**Saturday Feb. 18, 2023:** The data set being used has the information of banking clients. The data collected is their age, job, marital status, education, if the client is in default, their balance with the bank, if they own a house, currently have a loan from the bank, what day of the month they were contacted , which month, the duration of the call in seconds, if they have been part of a previous compaign, and if they made a term deposit with the bank. The goal of this project is classify when a client will make a term deposit with the bank. 

My objective is to build a model that maximizes the ratio  True positive/(True Positive + False Positive), i.e. we want to maximize the return on investment of effort.


I ended following along with the code described in https://www.kaggle.com/code/manred1997/eda-preprocessing-modeling-evaluate-std.

**References**

S. Moro, P. Cortez and P. Rita. A Data-Driven Approach to Predict the Success of Bank Telemarketing. Decision Support Systems, Elsevier, 62:22-31, June 2014

https://www.kaggle.com/datasets/prakharrathi25/banking-dataset-marketing-targets

In [1]:
## For data handling
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

# import library for normalizing, modeling, evaluate
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, f1_score, auc
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

## This sets the plot style
sns.set_style("whitegrid")

In [2]:
df_test = pd.read_csv('/Users/tmpolstra/Dropbox/Python Projects/Classification_Projects/Banking_Datasets_Marketing_Targets/Project 1/test.csv', delimiter =';')
df_train = pd.read_csv('/Users/tmpolstra/Dropbox/Python Projects/Classification_Projects/Banking_Datasets_Marketing_Targets/Project 1/train.csv', delimiter =';')

In [3]:
df_train = df_train.drop(['day', 'month'], axis=1)
df_test = df_test.drop(['day', 'month'], axis=1)

In [5]:
for col in ['default', 'housing', 'loan', 'y']:
    df_train[col].replace(['yes', 'no'], [1, 0], inplace=True)
    df_test[col].replace(['yes', 'no'], [1, 0], inplace=True)

In [6]:
cat_cols = ['job', 'marital', 'education', 'poutcome']
num_cols = ['age', 'default', 'balance', 'housing', 'loan', 'duration', 'campaign', 'pdays', 'previous']
tar_cols = ['y']

In [7]:
df_cat = df_train[cat_cols + tar_cols]
df_num = df_train[num_cols + tar_cols]

In [8]:
# Correlation Test
#Checking to see if any of the variables are strongly correlated with one another.
plt.figure(figsize=(8,4))
mask = np.triu(np.ones_like(df_num.corr(), dtype=bool))
sns.heatmap(df_num.corr(), vmin=-1, vmax=1, mask=mask, annot=True)
plt.show()

TypeError: 'module' object is not callable

In [9]:
df_num[['age', 'balance', 'duration', 'campaign', 'previous']].skew()

age          0.684818
balance      8.360308
duration     3.144318
campaign     4.898650
previous    41.846454
dtype: float64

In [10]:
df_num[['age', 'balance', 'duration', 'campaign', 'previous']].kurtosis()

age            0.319570
balance      140.751547
duration      18.153915
campaign      39.249651
previous    4506.860660
dtype: float64

In [11]:
def plot_histogram(data, name, bins=30, kde=True, figsize=(8,4)):
    plt.figure(figsize=figsize)
    sns.set_theme(style="whitegrid")
    sns.histplot(x=data[name], data=data, hue='y',kde=kde)
    # plt.title(name)
    plt.show()

    plt.figure(figsize=figsize)
    sns.set_theme(style="whitegrid")
    sns.histplot(x=data[name], data=data, kde=kde)
    # plt.title(name)
    plt.show()


    plt.figure(figsize=figsize)
    sns.set_theme(style="whitegrid")
    sns.kdeplot(x=data[name], data=data, cumulative=True)
    # plt.title(name)
    plt.show()

In [12]:
for col in ['age', 'balance', 'duration', 'campaign', 'previous']:
    plot_histogram(df_train, col)

TypeError: 'module' object is not callable

In [39]:
x = df_num[df_num.columns[:-1]]
y = df_num[df_num.columns[-1]]

#x = MinMaxScaler().fit_transform(x)
f_score, p_value = f_classif(x, y)
for i in range(len(df_num.columns[:-1])):
    print(df_num.columns[:-1][i], ': ', p_value[i])

age :  8.825643692172049e-08
default :  1.866326526158345e-06
balance :  2.5211136918594683e-29
housing :  2.621926571845309e-194
loan :  9.793657535311501e-48
duration :  0.0
campaign :  1.0123473642045049e-54
pdays :  3.79055331945749e-108
previous :  7.801830349772383e-88


In [16]:
le = LabelEncoder()

In [17]:
x = df_cat[df_cat.columns[:-1]].copy()
y = df_cat[df_cat.columns[-1]]

for col in df_cat.columns[:-1]:
    x[col] = le.fit_transform(x[col])


In [ ]:
#x = MinMaxScaler().fit_transform(x)

In [18]:
chi_score, pvalue = chi2(x, y)

In [19]:
for i in range(len(df_cat.columns[:-1])):
    print(df_cat.columns[:-1][i], ': ', pvalue[i])

job :  1.4125763345435744e-41
marital :  4.8744975856275684e-08
education :  1.7429221644637613e-21
poutcome :  1.4372537073338641e-24


In [20]:
# Label Encoding
le = LabelEncoder()
df_train['job'] = le.fit_transform(df_train['job'])
df_test['job'] = le.fit_transform(df_test['job'])

In [21]:
df_train = pd.get_dummies(df_train, columns=['marital', 'poutcome', 'contact'])
df_test = pd.get_dummies(df_test, columns=['marital', 'poutcome', 'contact'])

In [22]:
df_train['education'].replace(['primary', 'secondary', 'tertiary', 'unknown'], [1, 2, 3, 0], inplace=True)
df_test['education'].replace(['primary', 'secondary', 'tertiary', 'unknown'], [1, 2, 3, 0], inplace=True)

In [23]:
# Standardize numerical data

scaler = StandardScaler()

cols = ['age', 'job', 'education', 'default', 'balance', 'housing', 'loan', 'duration', 'campaign', 'pdays', 'previous']

X_train = pd.DataFrame(scaler.fit_transform(df_train[cols]))
X_test = pd.DataFrame(scaler.transform(df_test[cols]))

Y_train = df_train['y']
Y_test = df_test['y']


X_train.columns = cols
X_test.columns = cols

In [24]:
X_train = pd.concat([X_train, df_train.iloc[:, len(cols)+1:]], axis=1)
X_test = pd.concat([X_test, df_test.iloc[:, len(cols)+1:]], axis=1)

In [25]:
X_test.head()

,age,job,education,default,balance,housing,loan,duration,campaign,pdays,...,marital_divorced,marital_married,marital_single,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown,contact_cellular,contact_telephone,contact_unknown
0,-1.029906,1.729573,-1.361914,-0.13549,0.139496,-1.118674,-0.436803,-0.695712,-0.569351,-0.411453,...,False,True,False,False,False,False,True,True,False,False
1,-0.747384,0.812877,-0.077715,-0.13549,1.125461,0.893915,2.289359,-0.148192,-0.569351,2.984213,...,False,True,False,True,False,False,False,True,False,False
2,-0.559037,-0.103820,1.206484,-0.13549,-0.004031,0.893915,-0.436803,-0.284101,-0.569351,2.894327,...,False,False,True,True,False,False,False,True,False,False
3,-1.029906,-0.103820,1.206484,-0.13549,0.037352,0.893915,2.289359,-0.229737,0.399020,-0.411453,...,False,True,False,False,False,False,True,False,False,True
4,1.701139,-1.020516,-0.077715,-0.13549,-0.447419,0.893915,-0.436803,-0.124893,-0.569351,-0.411453,...,False,True,False,False,False,False,True,False,False,True


In [26]:
train = pd.concat([X_train, Y_train], axis=1)

In [27]:
test = pd.concat([X_test, Y_test], axis=1)

In [28]:
X_train, Y_train = train.iloc[:, :-1], train.iloc[:, -1]
X_test, Y_test = test.iloc[:, :-1], test.iloc[:, -1]

In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

In [30]:
models = []
models.append(LogisticRegression(solver='liblinear'))
models.append(SVC())
models.append(KNeighborsClassifier())
models.append(DecisionTreeClassifier())
models.append(RandomForestClassifier())
models.append(GaussianNB())
model_list = ['Logistic Regression', 'SVM', 'KNN', 'Decision Tree', 'Random Forest', 'Naive Bayes']

In [31]:
acc_list = []
auc_list = []
cm_list = []

In [32]:
DTC = DecisionTreeClassifier()

In [33]:
DTC.fit(X_train,Y_train)

DecisionTreeClassifier()

In [34]:
metrics.accuracy_score(Y_test,DTC.predict(X_test))

1.0

In [36]:
for model in models:
    model.fit(X_train, Y_train)
    Y_pred = model.predict(X_test)
    acc_list.append(metrics.accuracy_score(Y_test, Y_pred))
    fpr, tpr, _thresholds = metrics.roc_curve(Y_test, Y_pred)
    auc_list.append(round(metrics.auc(fpr, tpr),2))
    cm_list.append(confusion_matrix(Y_test, Y_pred))

KeyboardInterrupt: 

In [38]:
result_df = pd.DataFrame({'Model':model_list, 'Accuracy': acc_list, 'AUC': auc_list})
result_df

ValueError: All arrays must be of the same length

In [37]:
fig = plt.figure(figsize = (18,10))
for i in range(len(cm_list)):
    cm = cm_list[i]
    model = model_list[i]
    sub = fig.add_subplot(2, 3, i+1).set_title(model)
    cm_plot = sns.heatmap(cm, annot=True, fmt="d", cmap = 'Blues_r')
    cm_plot.set_xlabel('Predicted Values')
    cm_plot.set_ylabel('Actual Values ')


TypeError: 'module' object is not callable

In [ ]:
from sklearn.metrics import roc_curve, auc, RocCurveDisplay

for model in models:
    RocCurveDisplay.from_estimator(model, X_test, Y_test)